In [1]:
seed = 12345
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import matplotlib 
matplotlib.use('Agg') 
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.models import load_model
import pandas as pd
import h5py
import numpy as np
import pickle
from PIL import ImageDraw
from PIL import Image
from PIL import ImageFont
from PIL.ExifTags import TAGS
import time

os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [4]:
def fix_image_orientation(img):
    try:
        if hasattr(img, '_getexif'): # only present in JPEGs
            for orientation in TAGS.keys(): 
                if TAGS[orientation]=='Orientation':
                    break 
            e = img._getexif()       # returns None if no EXIF data
            if e is not None:
                exif=dict(e.items())
                orientation = exif[orientation]

                if orientation == 3:   img = img.transpose(Image.ROTATE_180)
                elif orientation == 6: img = img.transpose(Image.ROTATE_270)
                elif orientation == 8: img = img.transpose(Image.ROTATE_90)
    except:
        pass
    return img

#### The script used to save food info into a pickle file from a csv tab txt file
<code>
import csv
file = csv.reader(open("./UECFOOD100/category_calorie.txt"))
food_info = {}
for row in file:
    line = row[0].split('\t')
    food_info[line[0]] = (line[1].capitalize(), int(line[2]))

with open('food_info.p', 'wb') as fp:
    pickle.dump(food_info, fp)

</code>

In [5]:
# Load the class indices map
import pickle
with open('class_indices_map.p', 'rb') as fp:
    prediction_to_class_map = pickle.load(fp)

In [6]:
# Load food class info: Name, and Calorie.
# Dictionary: food_info[str class] -> (str Name, int calories)
with open('food_info.p', 'rb') as fp:
    food_info = pickle.load(fp)

In [7]:
# Load the original trained model with its weights
model = load_model('fine_tuned_model_all.h5')

In [8]:
# Load the fine-tuned weights
model.load_weights('weights.best_28_11.hdf5')

In [113]:
# train_data_dir = './UECFOOD100/data/train'
# validation_data_dir = './UECFOOD100/data/validation'
# img_width, img_height = 150, 150
# nb_train_samples = 10183
# nb_validation_samples = 4428

In [114]:
# test_datagen = ImageDataGenerator(rescale=1./255)

# validation_generator = test_datagen.flow_from_directory(
#         validation_data_dir,
#         target_size=(img_height, img_width),
#         batch_size=1,
#         class_mode='categorical',
#         shuffle=False,
#         seed=seed)

In [11]:
repeat_duration = 600 #seconds
pause_time = 3
loops = repeat_duration/pause_time
print('Started monitoring folder for new images...')
while 1:
    if loops < 0: break
    loops -= 1
    #time.asctime(time.localtime(time.time()))
    time.sleep(pause_time)
    allfiles = os.listdir(r'C:\Users\ali_a\Dropbox')
    for f in allfiles:
        if f.endswith(".jpg") and f[:-4]+'_prediction.jpg' not in allfiles and 'prediction' not in f:
            img_path = os.path.join(r'C:\Users\ali_a\Dropbox',f)
            try:
                img = image.load_img(img_path, target_size=(150, 150))             
            except:
                print('file read error. Skipped')
                continue
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)

            prediction_proba = model.predict(x/255)            
            prediction = prediction_proba.argsort()[0,-1::-1]
            prediction = np.vectorize(prediction_to_class_map.get)(prediction)

            print (img_path[:-4]+'_prediction.jpg', prediction[0:6], food_info[prediction[0]][0], 'Contains',
                  food_info[prediction[0]][1], 'Calories.')

            # Save the result as a textbox on a new image ending with _prediction
            img = Image.open(img_path)
            img = fix_image_orientation(img)
            # Resize large images:
            if img.size[0] > 1000:
                ratio = img.size[0]/1000
                new_width = 1000
                new_height = int(img.size[1]/ratio)
                img = img.resize((new_width, new_height), resample=Image.BILINEAR)
            
            alt_img = img.copy()

            draw = ImageDraw.Draw(img)
            fontsize = 1
            font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)
            text = food_info[prediction[0]][0]+ '\n' + str(food_info[prediction[0]][1]) + ' calories.'
            
            while draw.textsize(text, font=font)[0] < 0.35*img.size[0]:
                fontsize +=1
                font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)

            fontsize -= 1
            if fontsize < 12:
                fontsize = 12
            font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)
            textsize = draw.textsize(text, font=font)
            offset = font.getoffset(text)

            draw.rectangle((0,0,textsize[0]+offset[0],textsize[1]+offset[1]), fill=(255,255,255))
            draw.text((0, 0),text,(0,0,0),font=font)

            output_path = img_path[:-4]+'_prediction.jpg'
            img.save(output_path)
            
            ## Alternative image with top-5 predictions
            draw = ImageDraw.Draw(alt_img)
            fontsize = 1
            font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)
            alt_text = '\n'.join([' '.join([str(food_info[_][0]), str(food_info[_][1]), 'Calories']) for _ in prediction[0:5]][0:6])
            while draw.textsize(alt_text, font=font)[0] < 0.3*alt_img.size[0]:
                fontsize +=1
                font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)

            fontsize -= 1
            if fontsize < 12:
                fontsize = 12
            font = ImageFont.truetype(r'C:\Windows\Fonts\ARIAL.TTF', fontsize)
            textsize = draw.textsize(alt_text, font=font)
            offset = font.getoffset(alt_text)

            draw.rectangle((0,0,textsize[0]+offset[0],textsize[1]+offset[1]), fill=(255,255,255))
            draw.text((0, 0),alt_text,(0,0,0),font=font)

            output_path = img_path[:-4]+'_top5_predictions.jpg'
            alt_img.save(output_path)

print('Stopped!')

Started monitoring folder for new images...
C:\Users\ali_a\Dropbox\IMG_20161215_124547_prediction.jpg ['6' '43' '92' '59' '9' '3'] Beef curry Contains 520 Calories.
Stopped!
